In [1]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
from scipy.cluster.vq import vq,kmeans
from sklearn.preprocessing import StandardScaler


In [6]:
#Step 1
#Choose three different categories of objects from the Caltech 101 dataset, as diverse from each other as possible.

In [3]:
def read_dataset_metadata(dir_path):
    dataset = {}
    for filename in os.listdir(dir_path):
        if os.path.isdir(dir_path + filename):
            dataset[filename] = []
            for img in os.listdir(os.path.join(dir_path + filename)):
                dataset[filename].append(os.path.join(dir_path,filename,img))
    return dataset

def get_frequent_object(dataset,count=3):
    """
    dataset : Dictionary object returned by the function 'read_dataset'
    
    Returns a list of objects with their frequency in descending order
    """
    key_count = {}
    for key in dataset.keys():
        key_count[key] = len(dataset[key])
    
    
    object_keys = []
    for key, value in sorted(key_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
        object_keys.append(key)
        count-=1
        if count == 0:
            break
    return object_keys
    

In [4]:
dataset_metadata = read_dataset_metadata('./101_ObjectCategories/')

In [5]:
frequent_objects = get_frequent_object(dataset_metadata,5)
print frequent_objects

['airplanes', 'Motorbikes', 'BACKGROUND_Google', 'Faces_easy', 'Faces']


In [6]:
#The output of the above cell shows the 5 most frequent objects
#In the interest of choosing three different categories as diverse from each other as possible
#We choosse 'airplanes', 'Motorbikes' and 'Faces_easy' classes

In [7]:
categories = ['airplanes', 'Motorbikes','Faces_easy']

new_dataset_metadata = {}
for category in categories:
    new_dataset_metadata[category] = dataset_metadata[category]

In [61]:
def make_dataset(metadata):
    try:
        dataset = []
        for key in metadata.keys():
            for img_path in metadata[key]:
                img = cv2.imread(img_path,0)
                dataset.append((img,key))
        dataset = np.array(dataset)
        np.save('dataset',dataset)
        return 'True
    except:
        raise "Error in creating Dataset File"

In [62]:
make_dataset(new_dataset_metadata)

In [9]:
dataset = np.load('dataset.npy')

In [10]:
"""
Step 2
Extract some local features (SIFT/SURF), cluster them using k-Means algorithm, and
create a bag-of-words representation for images. This bag-of-word representation is to be
used as image feature in the subsequent steps.
"""

'\nStep 2\nExtract some local features (SIFT/SURF), cluster them using k-Means algorithm, and\ncreate a bag-of-words representation for images. This bag-of-word representation is to be\nused as image feature in the subsequent steps.\n'

In [94]:
def make_descriptor_data(dataset):
    try:
        descriptor_list = []
        sift = cv2.xfeatures2d.SIFT_create()
        for img_data in dataset:
            img = img_data[0]
            label = img_data[1]
            #Computing descriptors for every image
            kpts = sift.detect(img)
            kpts, des = sift.compute(img, kpts)
            descriptor_list.append((img, des, label))
            
        descriptor_list = np.array(descriptor_list)
        np.save('descriptor_data',descriptor_list)
        return True
    except:
        raise "Error in creating Bag of Words File"

In [95]:
make_descriptor_data(dataset)

True

In [99]:
descriptor_data = np.load('descriptor_data.npy')

In [100]:
#Stacking descriptors before clustering
descriptors = descriptor_data[0][1]
for img, descriptor,label in descriptor_data[1:]:
    descriptors = np.vstack((descriptors, descriptor))  


In [101]:
#K-Means clustering of the descriptors
k = 100
voc, variance = kmeans(descriptors, k, 1) 

In [125]:
#Crearing Histogram of the Bag of Words
im_features = np.zeros((len(dataset), k), "float32")
class_labels =[]
for i in xrange(len(dataset)):
    words, distance = vq(descriptor_data[i][1],voc)
    class_labels.append(descriptor_data[i][2])
    for w in words:
        im_features[i][w] += 1
        
stdSlr = StandardScaler(with_mean=False).fit(im_features)
im_features = stdSlr.transform(im_features)
class_labels = np.array(class_labels)

In [135]:
feature_dataset = []
for im_feature,label in zip(im_features,class_labels):
    feature_dataset.append((im_feature,label))

feature_dataset=np.array(feature_dataset)
np.save('bag_of_words_features',feature_dataset)

In [136]:
feature_dataset = np.load('bag_of_words_features.npy')